In [61]:
import json

In [62]:
from openai import OpenAI

In [63]:
from elasticsearch import Elasticsearch

In [64]:
with open("../../documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [65]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [66]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [67]:
q = 'the course has already started, can I still enroll?'

In [68]:
client = OpenAI()

In [69]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on several factors, including the policies of the institution offering the course, the specific course structure, and availability. Here are some steps you can take:\n\n1. **Contact the Institution:** Reach out to the admissions office, registrar, or the department offering the course to inquire about late enrollment policies.\n\n2. **Instructor Approval:** In some cases, you may need to obtain permission from the course instructor to join after the start date.\n\n3. **Understand Implications:** If late enrollment is permitted, make sure you understand any implications, such as missed assignments, lectures, or exams, and determine how you can catch up.\n\n4. **Consider Alternatives:** If late enrollment is not possible, inquire about the next available session or similar courses that might still be open for enrollment.\n\n5. **Review Refund Policies:** If you're enrolling late, check the institution's refund or 

In [70]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [71]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [72]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [73]:
rag(query)

'To run Kafka, specifically a Java Kafka producer/consumer/kstreams, navigate to the project directory and execute the following command in the terminal:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your jar file.'

In [74]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even if it has already started. Although you may not be able to submit some of the homework, you are still eligible to participate in the course. To obtain a certificate, you need to submit 2 out of 3 course projects and review 3 peers' projects by the deadline. If you join by the end of November and complete two projects, you can still qualify for a certificate."

In [75]:
es_client = Elasticsearch('http://localhost:9200')

In [76]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [77]:
from tqdm.auto import tqdm

In [78]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [79]:
query = 'I just disovered the course. Can I still join it?'

In [80]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [81]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [82]:
rag(query)

'Yes, you can still join the course even if it has already started. You are eligible to submit the homework assignments, but make sure to keep track of the deadlines for the final projects to avoid any last-minute issues.'